# Text generation with an RNN

### Learning Objectives

- Learn how to generate text using a RNN
- Create training examples and targets for text generation
- Build a RNN model for sequence generation using Keras Subclassing
- Create a text generator and evaluate the output

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Below is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but here are some things to consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## 1. Setup

### 1a. Import TensorFlow and other libraries

In [1]:
import os
import time

import numpy as np
import tensorflow as tf

### 1b. Explore the Shakespeare dataset

**KB Notes:** The origin file will be downloaded to the cache dir `~/.keras/datasets`.

In [2]:
# Download origin file
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)

1130496/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(text[:250]) # First 250 characters in text

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
print(f"Length of text: {len(text)} characters")

vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

Length of text: 1115394 characters
65 unique characters


## 2. Process the text

### 2a. Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

Split the text into tokens first. Then the `tf.keras.layers.StringLookup` layer can convert each character token into a numeric ID.

In [5]:
# Create a lookup layer with a list of known vocabulary (the 65 unique characters)
convert_chars_to_ids = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

example_texts = ["abcdefg", "xyz"] # Data

# Split the example data into tokens (a list of character sequences)
chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
print(f"Character Tokens: {chars}")

# Convert (vectorize) tokens to character IDs
ids = convert_chars_to_ids(chars)
print(f"Character IDs: {ids}")

Character Tokens: <tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>
Character IDs: <tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>


2023-07-08 01:17:33.616149: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-08 01:17:33.715415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-08 01:17:33.717083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-08 01:17:33.720794: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Since we want to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this, we can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Instead of passing the original vocabulary generated with `sorted(set(text))`, use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens are set the same way.

In [6]:
# Create a lookup layer that recovers human-readable strings from the vector representation
convert_ids_to_chars = tf.keras.layers.StringLookup(
    vocabulary=convert_chars_to_ids.get_vocabulary(), invert=True, mask_token=None
)

# Convert the character IDs back to tokens
chars = convert_ids_to_chars(ids)
print(f"Character Tokens: {chars}")

# Join the characters back into strings
def convert_ids_to_text(ids_param):
    return tf.strings.reduce_join(convert_ids_to_chars(ids_param), axis=-1)

strings = convert_ids_to_text(ids).numpy()
print(f"Strings: {strings}")


Character Tokens: <tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>
Strings: [b'abcdefg' b'xyz']


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### 2b. Create training examples and targets

Divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

In [7]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)
print(f"Examples per epoch: {examples_per_epoch}")

Examples per epoch: 11043


In [8]:
# Split the source text into a list of characters and vectorize them into character IDs
text_chars = tf.strings.unicode_split(text, input_encoding="UTF-8")
all_ids = convert_chars_to_ids(text_chars)
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [9]:
# Convert the text vector into a stream of character indices
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [10]:
for ids in ids_dataset.take(10):
    print(convert_ids_to_chars(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i
t
i


In [11]:
# Convert these individual characters into sequences of the desired size
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(seq)

tf.Tensor(
[19 48 57 58 59  2 16 48 59 48 65 44 53 11  1 15 44 45 54 57 44  2 62 44
  2 55 57 54 42 44 44 43  2 40 53 64  2 45 60 57 59 47 44 57  7  2 47 44
 40 57  2 52 44  2 58 55 44 40 50  9  1  1 14 51 51 11  1 32 55 44 40 50
  7  2 58 55 44 40 50  9  1  1 19 48 57 58 59  2 16 48 59 48 65 44 53 11
  1 38 54 60  2], shape=(101,), dtype=int64)


In [12]:
# To better see what's happening, convert them back to tokens, and join the tokens back into strings
for seq in sequences.take(1):
    print(convert_ids_to_chars(seq))
    
for seq in sequences.take(5):
    print(convert_ids_to_text(seq).numpy())

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)
b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be

For training, we'll need a dataset of `(input, label)` pairs, where `input` and 
`label` are sequences. At each timestep, the input is the current character and the label is the next character.

In [13]:
# Takes a sequence, duplicates it, then shifts it to align the input and label for each timestep
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [14]:
# Example output of splitting a sequence and shifting
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [15]:
# For each sequence, create (input, label) pairs
dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in dataset.take(5):
    print("Input :", convert_ids_to_text(input_example).numpy())
    print("Target:", convert_ids_to_text(target_example).numpy())
    print("---")

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
---
Input : b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
Target: b're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
---
Input : b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us k"
Target: b"ow Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
---
Input : b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be "
Target: b"l him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
---
Input : b'one: away, away!\n\nSecond Citiz

2023-07-08 01:17:36.522488: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


### 2c. Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [17]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## 3. Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

### 3a. Build a model with the following layers

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (We can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [18]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

The class below does the following:
- We derive a class from tf.keras.Model
- The constructor is used to define the layers of the model
- We define the pass forward using the layers defined in the constructor

In [19]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        # Create an embedding layer (args: input dim, output dim)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        # Create a GRU layer (args: units)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        
        # Finally connect it with a dense layer (args: units)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        # Because we are training a text generation model, use the previous state.
        # If there is no state, initialize the state.
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [20]:
model = MyModel(
    # Make sure that the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(convert_chars_to_ids.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

For each character, the model: 
1. looks up the embedding, 
2. runs the GRU one timestep with the embedding as input, and
3. applies the dense layer to generate logits predicting the log-likelihood of the next character.

## 4. Try the model

First, check the shape of the output:

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )

2023-07-08 01:17:40.014352: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8200


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [22]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  16896     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  67650     
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model, we need to sample from the output distribution in order to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [23]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0], num_samples=1
)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [24]:
sampled_indices

array([ 1, 40, 20, 51, 49, 15, 31, 14, 12,  8, 36, 44,  7,  3,  3, 38,  8,
        4, 53, 50, 65, 27, 60, 21, 52,  3, 61, 35, 13, 38, 44, 40, 43, 40,
       31,  8, 32, 59, 50,  6, 36, 23, 45, 38, 48,  3,  2, 53, 35, 29, 37,
       34, 11, 64,  0, 55, 22, 52, 25, 58, 32, 43, 50, 62, 11, 63, 29,  4,
       65, 63, 16,  8, 53, 55, 21, 35,  0, 53, 26, 16, 14, 12, 35, 18, 32,
       19, 35, 48, 49, 32, 19, 40, 55, 34,  5, 25, 31, 57, 26, 25])

Decode these to see the text predicted by this untrained model:

In [25]:
print("Input:\n", convert_ids_to_text(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", convert_ids_to_text(sampled_indices).numpy())

Input:
 b"d your affections are\nA sick man's appetite, who desires most that\nWhich would increase his evil. He"

Next Char Predictions:
 b"\naGljBRA;-We,!!Y-$nkzNuHm!vV?YeadaR-Stk'WJfYi! nVPXU:y[UNK]pImLsSdkw:xP$zxC-npHV[UNK]nMCA;VESFVijSFapU&LRrML"


## 5. Train the model

At this point, the problem can be treated as a standard classification problem. Given the previous RNN state and the input at this time step, predict the class of the next character.

### 5a. Attach an optimizer and a loss function
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because the model returns logits, we need to set the `from_logits` flag.


In [26]:
# Create a loss function
# For some reason, tf.keras.losses.sparse_categorical_crossentropy doesn't work here
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [27]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190453, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself: the output logits should all have similar magnitudes. To confirm this, we can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers and that it is badly initialized:

In [28]:
tf.exp(example_batch_mean_loss).numpy()

66.05271

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [29]:
model.compile(optimizer="adam", loss=loss)

### 5b. Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

### 5c. Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [40]:
EPOCHS = 30

In [41]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 11s 57ms/step - loss: 1.1562
Epoch 2/30
172/172 [==============================] - 12s 58ms/step - loss: 1.1142
Epoch 3/30
172/172 [==============================] - 12s 59ms/step - loss: 1.0711
Epoch 4/30
172/172 [==============================] - 12s 59ms/step - loss: 1.0248
Epoch 5/30
172/172 [==============================] - 12s 60ms/step - loss: 0.9759
Epoch 6/30
172/172 [==============================] - 12s 60ms/step - loss: 0.9258
Epoch 7/30
172/172 [==============================] - 12s 61ms/step - loss: 0.8723
Epoch 8/30
172/172 [==============================] - 12s 60ms/step - loss: 0.8207
Epoch 9/30
172/172 [==============================] - 12s 61ms/step - loss: 0.7698
Epoch 10/30
172/172 [==============================] - 12s 61ms/step - loss: 0.7233
Epoch 11/30
172/172 [==============================] - 12s 61ms/step - loss: 0.6785
Epoch 12/30
172/172 [==============================] - 12s 61ms/step - loss: 0.6392
E

#### Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

Each time the model is called, we pass in some text and an internal state. Then the model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

The following makes a single step prediction:

In [42]:
class OneStep(tf.keras.Model):
    def __init__(self, model, convert_ids_to_chars, convert_chars_to_ids, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.convert_ids_to_chars = convert_ids_to_chars
        self.convert_chars_to_ids = convert_chars_to_ids

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.convert_chars_to_ids(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(convert_chars_to_ids.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.convert_chars_to_ids(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits] 
        # inputs, states=None, return_state=False, training=False
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True, training=False)
        
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.convert_ids_to_chars(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [43]:
one_step_model = OneStep(model, convert_ids_to_chars, convert_chars_to_ids)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [44]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
How nought so, King Richard! where have you beet
Wagaret close unto his rescues;
And therefore fortify your knees and mark!

First Watchman:
Here is my sharp wretched gates of damned sor,
A pair of sincer:
Rather than other bodies as ease
At me in many thousand times,
And livident missly but to say home,
From where I stay with me another for along;
And never forbid the outwaft for his hands that I should
winning herminy.

HERMIONE:
But 'tis, here comes your both;
To wherefore were you both. Therefore process by son
To throw this worthy words that wounds me wrong,
Gives in the admired beauties of Rome,
crafts, on much great men that accounted at this health.
Therefore, she none well else of daling rise;
I am the modness of their maiden lorg,
Sull you the winds of Cares of me, whose eyes, call'd,
And so in prison and many proud me.
Mistress Kate tell him now, frier in death.

Provost:
God for every county!
O monstrous! and therefore, be still before.

KING LEWIS XI:
But, Wapwicate

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [45]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
What say'st thou? marry, both be, to
make lament of emptrone his valour and
displeasure of thy motter's, hung for flunks
To their hell chaste, 'twixt us thanks:
Revolvise great Auteous break that cares his brother
Cault our brothers and Kate.

HASTINGS:
I thought of wills and five.

GENZALO:
And frown, or else among the limbs and twict
As all thy dog, sits all together air, thy ship about
The public taking are the Lord Aumerle, Lucentio is she.

KATHARINA:
Our countryman,
The prich levier and the chairs of heaven
Belonged to the world; all great are pilly,
But 'manion'd, he does accuse me; Frolarks!
What duty as these black magistrates, and death so tend;
And that so play the efertive balm of the
folly?

Third Servingman:
Talk not, I say!

LUCIO:
Ah, Clifford, title when I mean to frame,
I'll mean, a book of prayer and my knightly grow:
I'll try it not so several tongues there.

ROMEO:
And that my boy have not done with this ourteems:
I'll not be the meeting of the place; some o

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [46]:
tf.saved_model.save(one_step_model, "one_step")
one_step_reloaded = tf.saved_model.load("one_step")

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [47]:
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
There is no hope it. Wherefore? what life?

THOMAS MOWBRAY:
O, let my sovereign turn away his way:



## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {"loss": loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [ ]:
model.fit(dataset, epochs=1)

Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for batch_n, (inp, target) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs["loss"])

        if batch_n % 50 == 0:
            template = (
                f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            )
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f"Epoch {epoch+1} Loss: {mean.result().numpy():.4f}")
    print(f"Time taken for 1 epoch {time.time() - start:.2f} sec")
    print("_" * 80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))